In [49]:
import sys
 
sys.path.append('..')
from piece import piece

piece.start(should_send_to_score=False)

In [ ]:
from soundmining_tools.supercollider_receiver import ExtendedNoteHandler, PatchArguments
from soundmining_tools.supercollider_client import SupercolliderClient
from soundmining_tools import note, spectrum
from soundmining_tools.generative import *
from soundmining_tools.sieve import *
from soundmining_tools.modular.control_instruments import ControlInstrument
from soundmining_tools.modular.instrument import NodeId
import math
from typing import Callable
from soundmining_tools.sequencer import Sequencer, SequenceNote
from enum import Enum
from soundmining_tools import supercollider_client
from soundmining_tools.modular import instrument
import random
from soundmining_tools.ui.ui_piece import UiPieceBuilder
from soundmining_tools.sequencer import Sequencer, SequenceNote
from ipycanvas import Canvas 

sound_path = "/Users/danielstahl/Documents/Music/Pieces/Ambient Music/Ambient Music 8/sounds/Ambient Music 8 Sounds"

# Guitar Strum 2-Omni mix.flac guitar_strum_thin_omni 124, 196, 258, 375, 515, 747, 984, 1217, 1521, 1709, 1902 
# Guitar Strum 2-Cardioid mix.flac guitar_strum_thin_cardioid 48, 60, 92, 125, 169, 232, 286, 377, 492, 679, 817, 1224, 1495, 1727, 1921

OMNI_SOUND = "guitar_strum_thin_omni"
CARDIOID_SOUND = "guitar_strum_thin_cardioid"

piece.reset()
piece.synth_player \
    .add_sound(OMNI_SOUND, f"{sound_path}/Guitar Strum 2-Omni mix.flac", 0.052, 0.535) \
    .add_sound(CARDIOID_SOUND, f"{sound_path}/Guitar Strum 2-Cardioid mix.flac", 0.052, 0.535) \
    .start()

amp_transition_matrix = {
    0: {1: 0.5, 2: 0.5},
    1: {0: 0.5, 2: 0.5},
    2: {0: 0.5, 1: 0.5}
}

amp_control_funcs = [
    lambda low, high: piece.control_instruments.three_block_control((0, random_range(low, high), random_range(low, high), 0), (0.1, 0.5, 0.4), (0, 0, 0)),
    lambda low, high: piece.control_instruments.three_block_control((0, random_range(low, high), random_range(low, high), 0), (0.4, 0.5, 0.1), (0, 0, 0)),
    lambda low, high: piece.control_instruments.three_block_control((0, random_range(low, high), random_range(low, high), 0), (0.4, 0.2, 0.4), (0, 0, 0))
]

class NoteState:
    def __init__(self, freq_transion_matrix: dict, freqs: list[float], amp: dict = None) -> None:
        self._freq_chain = MarkovChain(freq_transion_matrix, 0)
        self._freqs = freqs
        if amp:
            self._amp = MarkovChain(amp, 0)

    def next_freq(self) -> float:
        return self._freqs[self._freq_chain.next()]
    
    def next_amp(self, low: float=0.75, high: float=1.5) -> ControlInstrument:
        return amp_control_funcs[self._amp.next()](low, high)


low_omni_transition_matrix = {
    0: {1: 0.5, 2: 0.5},
    1: {0: 0.5, 2: 0.5},
    2: {0: 0.5, 1: 0.5}
}

SUB_NOISE_TRACK = "Sub noise"
SUB_REVERB_TRACK = "Sub reverb"
SUB_SOUND_TRACK = "Sub sound"
LOW_NOISE_TRACK = "Low noise"
LOW_REVERB_TRACK = "Low reverb"
LOW_SOUND_TRACK = "Low sound"
MIDDLE_NOISE_TRACK = "Middle noise"
MIDDLE_REVERB_TRACK = "Middle reverb"
MIDDLE_SOUND_TRACK = "Middle sound"
HIGH_NOISE_TRACK = "High noise"
HIGH_REVERB_TRACK = "High reverb"
HIGH_SOUND_TRACK = "High sound"
OMNI_COLOR = "red"
CARDIOD_COLOR = "blue"
OMNI_DURATION = piece.synth_player.get_sound(OMNI_SOUND).duration(1.0)
CARDIOID_DURATION = piece.synth_player.get_sound(CARDIOID_SOUND).duration(1.0)

#EFFECT_LENGTH = 30
EFFECT_LENGTH = 650

effect_sound = piece.synth_player.note(NodeId.EFFECT) \
    .stereo_input() \
    
reverb_effect_sound = piece.synth_player.note(NodeId.ROOM_EFFECT) \
    .input_from_note(effect_sound) \
    .stereo_volume(piece.control_instruments.static_control(0.2)) \
    .stereo_g_verb(piece.control_instruments.three_block_control((0, 1, 1, 0), (0.01, 0.90, 0.09), (0, 0, 0)), roomsize=30, revtime=10, damping=0.7, inputbw=0.5, earlyreflevel=0.5, taillevel=0.7) \
    .stereo_high_pass_filter(piece.control_instruments.static_control(100)) \
    .stereo_low_pass_filter(piece.control_instruments.static_control(700))
reverb_effect_sound.play(0, EFFECT_LENGTH, output_bus=0)

clean_effect_sound = piece.synth_player.note(NodeId.ROOM_EFFECT) \
    .input_from_note(effect_sound) \
    .stereo_volume(piece.control_instruments.static_control(0.8))
clean_effect_sound.play(0, EFFECT_LENGTH, output_bus=0)


low_omni_freqs = [124, 196, 258]
low_omni_long_state = NoteState(low_omni_transition_matrix, low_omni_freqs, amp_transition_matrix)

def low_omni_noise(start_time: float, base_duration: float) -> SequenceNote: 
    amp_control = low_omni_long_state.next_amp()
    pan_start, pan_end = pan_line(0.5, [(-0.5, -0.25), (0.25, 0.5)])    
    band_pass = low_omni_long_state.next_freq()
    rq = random_range(0.01, 0.05)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 196, 258)
    duration = base_duration * random_range(0.85, 1.15)
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .play(start_time + random_range(0.1, 0.5), duration)
    return SequenceNote(start=start_time, track=LOW_NOISE_TRACK, duration=duration, freq=band_pass, color=OMNI_COLOR)

low_omni_state = NoteState(low_omni_transition_matrix, low_omni_freqs)

def low_omni(start_time: float) -> SequenceNote:
    freq = low_omni_state.next_freq()
    pan = pan_point([(-0.5, -0.25), (0.25, 0.5)])    
    piece.synth_player.note() \
        .sound_mono(OMNI_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(3.85, 4.15))) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(freq)) \
        .pan(piece.synth_player.control_instruments.static_control(pan)) \
        .send_to_synth_note(effect_sound, start_time=start_time + random_range(0.01, 0.05))
        #.play(start_time + random_range(0.01, 0.05))
    return SequenceNote(start=start_time, track=LOW_SOUND_TRACK, duration=OMNI_DURATION, freq=freq, color=OMNI_COLOR)

def low_omni_reverb(start_time: float, base_duration: float) -> SequenceNote:
    duration = base_duration * random_range(0.85, 1.15)
    pan = pan_point([(-0.5, -0.25), (0.25, 0.5)])    
    freq = low_omni_long_state.next_freq()
    piece.synth_player.note() \
        .sound_mono(OMNI_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(3.85, 4.15))) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(freq)) \
        .pan(piece.synth_player.control_instruments.static_control(pan)) \
        .stereo_g_verb(piece.synth_player.control_instruments.sine_control(0, 1), roomsize=50, revtime=duration) \
        .play(start_time + random_range(0.01, 0.05), duration=duration)
    return SequenceNote(start=start_time, track=LOW_REVERB_TRACK, duration=duration, freq=freq, color=OMNI_COLOR)


sub_cardioid_transition_matrix = {
    0: {1: 0.5, 2: 0.5},
    1: {0: 0.5, 2: 0.5},
    2: {0: 0.5, 1: 0.5}
}
sub_cardioid_freqs = [48, 60, 92] 

sub_cardiod_long_state = NoteState(sub_cardioid_transition_matrix, sub_cardioid_freqs, amp_transition_matrix)

def sub_cardioid_noise(start_time: float, base_duration: float) -> SequenceNote:
    band_pass = sub_cardiod_long_state.next_freq()
    amp_control = sub_cardiod_long_state.next_amp(3, 5)
    pan_start, pan_end = pan_line(0.2, [(-0.2, 0.2)])
    rq = random_range(0.01, 0.05)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 169, 286)
    duration = base_duration * random_range(0.85, 1.15)
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .play(start_time + random_range(0.1, 0.5), duration)
    return SequenceNote(start=start_time, track=SUB_NOISE_TRACK, duration=duration, freq=band_pass, color=CARDIOD_COLOR)

sub_cardioid_state = NoteState(sub_cardioid_transition_matrix, sub_cardioid_freqs)

def sub_cardioid(start_time: float) -> SequenceNote:    
    freq = sub_cardioid_state.next_freq()
    piece.synth_player.note() \
        .sound_mono(CARDIOID_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(8, 13))) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(freq)) \
        .pan(piece.synth_player.control_instruments.static_control(random_range(-0.2, 0.2))) \
        .send_to_synth_note(effect_sound, start_time=start_time + random_range(0.01, 0.05))
        #.play(start_time + random_range(0.01, 0.05))
    return SequenceNote(start=start_time, track=SUB_SOUND_TRACK, duration=CARDIOID_DURATION, freq=freq, color=CARDIOD_COLOR)
        
def sub_cardioid_reverb(start_time: float, base_duration: float) -> SequenceNote:
    duration = base_duration * random_range(0.85, 1.15)
    freq = sub_cardiod_long_state.next_freq()
    piece.synth_player.note() \
        .sound_mono(CARDIOID_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(5, 8))) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(freq)) \
        .pan(piece.synth_player.control_instruments.static_control(random_range(-0.2, 0.2))) \
        .stereo_g_verb(piece.synth_player.control_instruments.sine_control(0, 5), roomsize=50, revtime=duration) \
        .play(start_time + random_range(0.01, 0.05), duration=duration)
    return SequenceNote(start=start_time, track=SUB_REVERB_TRACK, duration=duration, freq=freq, color=CARDIOD_COLOR)
    
low_cardioid_transition_matrix = {
    0: {1: 0.33, 2: 0.33, 3: 0.33},
    1: {0: 0.33, 2: 0.33, 3: 0.33},
    2: {0: 0.33, 1: 0.33, 3: 0.33},
    3: {0: 0.33, 1: 0.33, 2: 0.33},
}
low_cardioid_freqs = [125, 169, 232, 286]

low_cardioid_long_state = NoteState(low_cardioid_transition_matrix, low_cardioid_freqs, amp_transition_matrix)

def low_cardioid_noise(start_time: float, base_duration: float) -> SequenceNote:
    band_pass = low_cardioid_long_state.next_freq()
    amp_control = low_cardioid_long_state.next_amp()
    pan_start, pan_end = pan_line(0.5, [(-0.5, -0.25), (0.25, 0.5)])
    rq = random_range(0.01, 0.05)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 169, 286)
    duration = base_duration * random_range(0.85, 1.15)
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(band_pass), piece.synth_player.control_instruments.static_control(rq)) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .play(start_time + random_range(0.1, 0.5), duration)
    return SequenceNote(start=start_time, track=LOW_NOISE_TRACK, duration=duration, freq=band_pass, color=CARDIOD_COLOR)

low_cardioid_state = NoteState(low_cardioid_transition_matrix, low_cardioid_freqs)

def low_cardioid(start_time: float) -> SequenceNote:    
    freq = low_cardioid_state.next_freq()
    pan = pan_point([(-0.5, -0.25), (0.25, 0.5)])
    piece.synth_player.note() \
        .sound_mono(CARDIOID_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(3.85, 4.15))) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(freq)) \
        .pan(piece.synth_player.control_instruments.static_control(pan)) \
        .send_to_synth_note(effect_sound, start_time=start_time + random_range(0.01, 0.05))
        #.play(start_time + random_range(0.01, 0.05))
    return SequenceNote(start=start_time, track=LOW_SOUND_TRACK, duration=CARDIOID_DURATION, freq=freq, color=CARDIOD_COLOR)

def low_cardioid_reverb(start_time: float, base_duration: float) -> SequenceNote:
    duration = base_duration * random_range(0.85, 1.15)
    freq = low_cardioid_long_state.next_freq()
    pan = pan_point([(-0.5, -0.25), (0.25, 0.5)])
    piece.synth_player.note() \
        .sound_mono(CARDIOID_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(3.85, 4.15))) \
        .mono_low_pass_filter(piece.synth_player.control_instruments.static_control(freq)) \
        .pan(piece.synth_player.control_instruments.static_control(pan)) \
        .stereo_g_verb(piece.synth_player.control_instruments.sine_control(0, 1), roomsize=50, revtime=duration) \
        .play(start_time + random_range(0.01, 0.05), duration=duration)
    return SequenceNote(start=start_time, track=LOW_REVERB_TRACK, duration=duration, freq=freq, color=CARDIOD_COLOR)

middle_omni_transition_matrix = {
    0: {1: 0.33, 2: 0.33, 3: 0.33},
    1: {0: 0.33, 2: 0.33, 3: 0.33},
    2: {0: 0.33, 1: 0.33, 3: 0.33},
    3: {0: 0.33, 1: 0.33, 2: 0.33},
}
middle_omni_freqs = [375, 515, 747, 984]

middle_omni_long_state = NoteState(middle_omni_transition_matrix, middle_omni_freqs, amp_transition_matrix)

def middle_omni_noise(start_time: float, base_duration: float) -> SequenceNote:
    freq = middle_omni_long_state.next_freq()
    duration = base_duration * random_range(0.85, 1.15)
    pan_start, pan_end = pan_line(0.5, [(-0.75, -0.5), (0.5, 0.75)])
    rq = random_range(0.01, 0.05)
    amp_control = middle_omni_long_state.next_amp(2, 3)

    high_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 258, 375)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 784, 984)                            
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_high_pass_filter(high_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(freq), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .play(start_time + random_range(0.05, 0.15), duration)
    return SequenceNote(start=start_time, track=MIDDLE_NOISE_TRACK, duration=duration, freq=freq, color=OMNI_COLOR)

middle_omni_state = NoteState(middle_omni_transition_matrix, middle_omni_freqs)

def middle_omni(start_time: float) -> SequenceNote:
    freq = middle_omni_state.next_freq()

    piece.synth_player.note() \
        .sound_mono(OMNI_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(3.85, 4.15))) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(258)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(freq), piece.synth_player.control_instruments.static_control(0.5)) \
        .pan(piece.synth_player.control_instruments.static_control(pan_point([(-0.5, -0.25), (0.25, 0.5)]))) \
        .send_to_synth_note(effect_sound, start_time=start_time + random_range(0.01, 0.05))
        #.play(start_time + random_range(0.01, 0.05))
    return SequenceNote(start=start_time, track=MIDDLE_SOUND_TRACK, duration=OMNI_DURATION, freq=freq, color=OMNI_COLOR)

def middle_omni_reverb(start_time: float, base_duration: float) -> SequenceNote:
    duration = base_duration * random_range(0.85, 1.15)
    freq = middle_omni_long_state.next_freq()
    pan = pan_point([(-0.5, -0.25), (0.25, 0.5)])

    high_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 258, 375)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 747, 984)                            

    sine_peak = random_range(2, 3) if freq < 747 else random_range(8, 13)

    rq = random_range(0.01, 0.05)
    piece.synth_player.note() \
        .sound_mono(OMNI_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(5, 8))) \
        .mono_high_pass_filter(high_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(freq), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .pan(piece.synth_player.control_instruments.static_control(pan)) \
        .stereo_g_verb(piece.synth_player.control_instruments.sine_control(0, sine_peak), roomsize=50, revtime=duration) \
        .play(start_time + random_range(0.01, 0.05), duration=duration)    
    return SequenceNote(start=start_time, track=MIDDLE_REVERB_TRACK, duration=duration, freq=freq, color=OMNI_COLOR)

middle_cardioid_transition_matrix = {
    0: {1: 0.33, 2: 0.33, 3: 0.33},
    1: {0: 0.33, 2: 0.33, 3: 0.33},
    2: {0: 0.33, 1: 0.33, 3: 0.33},
    3: {0: 0.33, 1: 0.33, 2: 0.33},
}

middle_cardioid_freqs = [377, 492, 679, 817]

middle_cardioid_long_state = NoteState(middle_cardioid_transition_matrix, middle_cardioid_freqs, amp_transition_matrix)

def middle_cardioid_noise(start_time: float, base_duration: float) -> SequenceNote:
    freq = middle_cardioid_long_state.next_freq()
    duration = base_duration * random_range(0.85, 1.15)
    pan_start, pan_end = pan_line(0.5, [(-0.75, -0.5), (0.5, 0.75)])
    rq = random_range(0.01, 0.05)
    amp_control = middle_cardioid_long_state.next_amp(2, 3)

    high_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 286, 377)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 679, 817)                            
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_high_pass_filter(high_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(freq), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .play(start_time + random_range(0.05, 0.15), duration)
    return SequenceNote(start=start_time, track=MIDDLE_NOISE_TRACK, duration=duration, freq=freq, color=CARDIOD_COLOR)

middle_cardioid_state = NoteState(middle_cardioid_transition_matrix, middle_cardioid_freqs)

def middle_cardioid(start_time: float) -> SequenceNote:
    n = middle_cardioid_state.next_freq()
    piece.synth_player.note() \
        .sound_mono(CARDIOID_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(3.85, 4.15))) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(286)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(n), piece.synth_player.control_instruments.static_control(0.5)) \
        .pan(piece.synth_player.control_instruments.static_control(pan_point([(-0.5, -0.25), (0.25, 0.5)]))) \
        .send_to_synth_note(effect_sound, start_time=start_time + random_range(0.01, 0.05))
        #.play(start_time + random_range(0.01, 0.05)) 
    return SequenceNote(start=start_time, track=MIDDLE_SOUND_TRACK, duration=CARDIOID_DURATION, freq=n, color=CARDIOD_COLOR)

def middle_cardioid_reverb(start_time: float, base_duration: float) -> SequenceNote:
    duration = base_duration * random_range(0.85, 1.15)
    freq = middle_cardioid_long_state.next_freq()
    pan = pan_point([(-0.5, -0.25), (0.25, 0.5)])

    high_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 286, 377)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 679, 817)                            

    sine_peak = random_range(13, 21)

    rq = random_range(0.01, 0.05)
    piece.synth_player.note() \
        .sound_mono(CARDIOID_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(5, 8))) \
        .mono_high_pass_filter(high_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(freq), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .pan(piece.synth_player.control_instruments.static_control(pan)) \
        .stereo_g_verb(piece.synth_player.control_instruments.sine_control(0, sine_peak), roomsize=50, revtime=duration) \
        .play(start_time + random_range(0.01, 0.05), duration=duration)   
    return SequenceNote(start=start_time, track=MIDDLE_REVERB_TRACK, duration=duration, freq=freq, color=CARDIOD_COLOR)

high_omni_transition_matrix = {
    0: {1: 0.33, 2: 0.33, 3: 0.33},
    1: {0: 0.33, 2: 0.33, 3: 0.33},
    2: {0: 0.33, 1: 0.33, 3: 0.33},
    3: {0: 0.33, 1: 0.33, 2: 0.33},
}

high_omni_freqs = [1217, 1521, 1709, 1902]

high_omni_long_state = NoteState(high_omni_transition_matrix, high_omni_freqs, amp_transition_matrix)

def high_omni_noise(start_time: float, base_duration: float) -> SequenceNote:
    freq = high_omni_long_state.next_freq()
    duration = base_duration * random_range(0.85, 1.15)
    pan_start, pan_end = pan_line(0.5, [(-0.75, -0.5), (0.5, 0.75)])
    rq = random_range(0.01, 0.05)
    amp_control = high_omni_long_state.next_amp(2, 3)

    high_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 984, 1217)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 1709, 1902)                            
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_high_pass_filter(high_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(freq), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .play(start_time + random_range(0.05, 0.15), duration)
    return SequenceNote(start=start_time, track=HIGH_NOISE_TRACK, duration=duration, freq=freq, color=OMNI_COLOR)

high_omni_state = NoteState(high_omni_transition_matrix, high_omni_freqs, amp_transition_matrix)

def high_omni(start_time: float) -> SequenceNote:
    n = high_omni_state.next_freq()
    piece.synth_player.note() \
        .sound_mono(OMNI_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(3.85, 4.15))) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(984)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(n), piece.synth_player.control_instruments.static_control(0.5)) \
        .pan(piece.synth_player.control_instruments.static_control(pan_point([(-0.75, -0.5), (0.5, 0.75)]))) \
        .send_to_synth_note(effect_sound, start_time=start_time + random_range(0.01, 0.05))
        #.play(start_time + random_range(0.01, 0.05))
    return SequenceNote(start=start_time, track=HIGH_SOUND_TRACK, duration=OMNI_DURATION, freq=n, color=OMNI_COLOR)

def high_omni_reverb(start_time: float, base_duration: float) -> SequenceNote:
    duration = base_duration * random_range(0.85, 1.15)
    freq = high_omni_long_state.next_freq()
    pan = pan_point([(-0.5, -0.25), (0.25, 0.5)])

    high_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 984, 1217)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 1709, 1902)                            

    sine_peak = random_range(8, 13)

    rq = random_range(0.01, 0.05)
    piece.synth_player.note() \
        .sound_mono(OMNI_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(5, 8))) \
        .mono_high_pass_filter(high_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(freq), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .pan(piece.synth_player.control_instruments.static_control(pan)) \
        .stereo_g_verb(piece.synth_player.control_instruments.sine_control(0, sine_peak), roomsize=50, revtime=duration) \
        .play(start_time + random_range(0.01, 0.05), duration=duration)   
    return SequenceNote(start=start_time, track=HIGH_REVERB_TRACK, duration=duration, freq=freq, color=OMNI_COLOR)

high_cardioid_transition_matrix = {
    0: {1: 0.33, 2: 0.33, 3: 0.33},
    1: {0: 0.33, 2: 0.33, 3: 0.33},
    2: {0: 0.33, 1: 0.33, 3: 0.33},
    3: {0: 0.33, 1: 0.33, 2: 0.33},
}

high_cardioid_freqs = [1224, 1495, 1727, 1921]

high_cardioid_long_state = NoteState(high_cardioid_transition_matrix, high_cardioid_freqs, amp_transition_matrix)

def high_cardioid_noise(start_time: float, base_duration: float) -> SequenceNote:
    freq = high_cardioid_long_state.next_freq()
    duration = base_duration * random_range(0.85, 1.15)
    pan_start, pan_end = pan_line(0.5, [(-0.75, -0.5), (0.5, 0.75)])
    rq = random_range(0.01, 0.05)
    amp_control = high_cardioid_long_state.next_amp(2, 3)

    high_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 817, 1224)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 1727, 1921)                            
    piece.synth_player.note() \
        .white_noise(amp_control) \
        .mono_high_pass_filter(high_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(freq), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .pan(piece.synth_player.control_instruments.line_control(pan_start, pan_end)) \
        .play(start_time + random_range(0.05, 0.15), duration)
    return SequenceNote(start=start_time, track=HIGH_NOISE_TRACK, duration=duration, freq=freq, color=CARDIOD_COLOR)

high_cardioid_state = NoteState(high_cardioid_transition_matrix, high_cardioid_freqs)

def high_cardioid(start_time: float) -> SequenceNote:
    n = high_cardioid_state.next_freq()            
    piece.synth_player.note() \
        .sound_mono(CARDIOID_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(3.85, 4.15))) \
        .mono_high_pass_filter(piece.synth_player.control_instruments.static_control(817)) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(n), piece.synth_player.control_instruments.static_control(0.5)) \
        .pan(piece.synth_player.control_instruments.static_control(pan_point([(-0.75, -0.5), (0.5, 0.75)]))) \
        .send_to_synth_note(effect_sound, start_time=start_time + random_range(0.01, 0.05))
        #.play(start_time + random_range(0.01, 0.05))                       
    return SequenceNote(start=start_time, track=HIGH_SOUND_TRACK, duration=CARDIOID_DURATION, freq=n, color=CARDIOD_COLOR)
            
def high_cardioid_reverb(start_time: float, base_duration: float) -> SequenceNote:
    duration = base_duration * random_range(0.85, 1.15)
    freq = high_cardioid_long_state.next_freq()
    pan = pan_point([(-0.5, -0.25), (0.25, 0.5)])

    high_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 817, 1224)
    low_pass_sine_osc = piece.synth_player.control_instruments.sine_osc_control(piece.synth_player.control_instruments.static_control(random_range(0.2, 0.5)), 1727, 1921)                            

    sine_peak = random_range(8, 13)

    rq = random_range(0.01, 0.05)
    piece.synth_player.note() \
        .sound_mono(CARDIOID_SOUND, 1, piece.synth_player.control_instruments.static_control(random_range(5, 8))) \
        .mono_high_pass_filter(high_pass_sine_osc) \
        .mono_band_pass_filter(piece.synth_player.control_instruments.static_control(freq), piece.synth_player.control_instruments.static_control(rq)) \
        .mono_low_pass_filter(low_pass_sine_osc) \
        .pan(piece.synth_player.control_instruments.static_control(pan)) \
        .stereo_g_verb(piece.synth_player.control_instruments.sine_control(0, sine_peak), roomsize=50, revtime=duration) \
        .play(start_time + random_range(0.01, 0.05), duration=duration)
    return SequenceNote(start=start_time, track=HIGH_REVERB_TRACK, duration=duration, freq=freq, color=CARDIOD_COLOR)


class NoiseNoteHandler(ExtendedNoteHandler):
    def __init__(self, client: SupercolliderClient) -> None:
        super().__init__(client)        

    def handle_note(self, patch_arguments: PatchArguments) -> None:
        match patch_arguments.octave:
            case 2:
                match patch_arguments.note:
                    case 0:
                        low_omni_noise(patch_arguments.start, 13)
                    case 1:
                        sub_cardioid_noise(patch_arguments.start, 13)
                    case 2:
                        low_cardioid_noise(patch_arguments.start, 13)
                    case 3:
                        middle_omni_noise(patch_arguments.start, 13)
                    case 4:
                        middle_cardioid_noise(patch_arguments.start, 13)
                    case 5: 
                        high_omni_noise(patch_arguments.start, 13)
                    case 6:
                        high_cardioid_noise(patch_arguments.start, 13)
            case 3:
                match patch_arguments.note:
                    case 0:
                        low_omni(patch_arguments.start)
                    case 1:
                        sub_cardioid(patch_arguments.start)      
                    case 2:
                        low_cardioid(patch_arguments.start)
                    case 3:
                        middle_omni(patch_arguments.start)
                    case 4:
                        middle_cardioid(patch_arguments.start)
                    case 5:
                        high_omni(patch_arguments.start)
                    case 6:
                        high_cardioid(patch_arguments.start)
            case 4:
                match patch_arguments.note:
                    case 0:
                        low_omni_reverb(patch_arguments.start, 13)
                    case 1:
                        sub_cardioid_reverb(patch_arguments.start, 13)
                    case 2:
                        low_cardioid_reverb(patch_arguments.start, 13)
                    case 3:
                        middle_omni_reverb(patch_arguments.start, 13)
                    case 4:
                        middle_cardioid_reverb(patch_arguments.start, 13)
                    case 5:
                        high_omni_reverb(patch_arguments.start, 13)
                    case 6: 
                        high_cardioid_reverb(patch_arguments.start, 13)
                               
    
my_handler = NoiseNoteHandler(piece.supercollider_client)
piece.receiver.set_note_handler(my_handler)                    

class NoteStart(Enum):
    EARLY = 1
    LATE = 2

class NoteBlend(Enum):
    ONLY_NOISE = 1
    ONLY_REVERB = 2
    NOISE_AND_REVERB = 3

class PlaySound(Enum):
    EARLY = 1
    LATE = 2
    PAUSE = 3

class NoteTypeSwitch(Enum):
    CARIDOID = 1
    OMNI = 2

class NoteState:
    def __init__(self, note_start_chain: MarkovChain[NoteStart], 
                 note_type_chain: MarkovChain[NoteStart], 
                 note_blend_chain: MarkovChain[NoteBlend], 
                 play_sound_chain: MarkovChain[PlaySound],
                 nr_of_play_sound_chanin: MarkovChain[int]) -> None:
        self._note_start_chain = note_start_chain
        self._note_type_chain = note_type_chain
        self._note_blend_chain = note_blend_chain
        self._play_sound_chain = play_sound_chain
        self._nr_of_play_sound_chain = nr_of_play_sound_chanin

    def get_note_type(self) -> NoteTypeSwitch:
        return self._note_type_chain.next()
    
    def get_note_blend(self) -> NoteBlend:
        return self._note_blend_chain.next()
    
    def get_play_sound(self) -> PlaySound:
        return self._play_sound_chain.next()
    
    def get_note_start(self, base_time: float) -> float:
        match self._note_start_chain.next():
            case NoteStart.LATE:
                variation = random_range(0.85, 1.15)
            case NoteStart.EARLY:
                variation = random_range(0.25, 0.5)
        return base_time * variation

    def get_nr_of_play_sound(self) -> int:
        return self._nr_of_play_sound_chain.next()
    
high_note_state = NoteState(
    note_start_chain= MarkovChain(
        transition_matrix= {
            NoteStart.EARLY: {NoteStart.EARLY: 0.2, NoteStart.LATE: 0.8},
            NoteStart.LATE: {NoteStart.EARLY: 0.5, NoteStart.LATE: 0.5}
        }, initial_value=NoteStart.LATE),
    
    note_type_chain = MarkovChain(
        transition_matrix = {
            NoteTypeSwitch.CARIDOID: {NoteTypeSwitch.CARIDOID: 0.7, NoteTypeSwitch.OMNI: 0.3},
            NoteTypeSwitch.OMNI: {NoteTypeSwitch.CARIDOID: 0.7, NoteTypeSwitch.OMNI: 0.3},
        }, initial_value=NoteTypeSwitch.CARIDOID),
        
    note_blend_chain= MarkovChain(
        transition_matrix = {
            NoteBlend.ONLY_NOISE: {NoteBlend.ONLY_NOISE: 0.5, NoteBlend.ONLY_REVERB: 0.2, NoteBlend.NOISE_AND_REVERB: 0.3},
            NoteBlend.ONLY_REVERB: {NoteBlend.ONLY_NOISE: 0.8, NoteBlend.ONLY_REVERB: 0, NoteBlend.NOISE_AND_REVERB: 0.2},
            NoteBlend.NOISE_AND_REVERB: {NoteBlend.ONLY_NOISE: 0.8, NoteBlend.ONLY_REVERB: 0.2, NoteBlend.NOISE_AND_REVERB: 0.0},
        }, initial_value=NoteBlend.ONLY_NOISE),
        
    play_sound_chain= MarkovChain(
        transition_matrix= {
            PlaySound.EARLY: {PlaySound.EARLY: 0.0, PlaySound.LATE: 0.05, PlaySound.PAUSE: 0.95},
            PlaySound.LATE: {PlaySound.EARLY: 0.05, PlaySound.LATE: 0.0, PlaySound.PAUSE: 0.95},
            PlaySound.PAUSE: {PlaySound.EARLY: 0.05, PlaySound.LATE: 0.05, PlaySound.PAUSE: 0.9},
        }, initial_value=PlaySound.PAUSE),
    nr_of_play_sound_chanin=MarkovChain(
        transition_matrix= {
            1: {1: 0.5, 2: 0.5},
            2: {1: 0.9, 2: 0.1},
        }, initial_value=1)
)

high_times = [(8, 13), (13, 13), (8, 13), (8, 13), (13, 13), (21, 13)] * 10

def high_step_handler(i: int, time: float) -> list[SequenceNote]:
    notes = []
    base_dur = high_times[i][1]
    if high_note_state.get_note_type() == NoteTypeSwitch.CARIDOID:
        match high_note_state.get_note_blend():
            case NoteBlend.ONLY_NOISE:
                notes.append(high_cardioid_noise(time, base_dur))
            case NoteBlend.ONLY_REVERB:
                notes.append(high_cardioid_reverb(time, base_dur))
            case NoteBlend.NOISE_AND_REVERB:
                notes.extend([
                    high_cardioid_noise(time, base_dur),
                    high_cardioid_reverb(time, base_dur)
                ])
        match high_note_state.get_play_sound():
            case PlaySound.EARLY:
                start_time = time + (3 * random_range(0.05, 0.15))
                nr_of_sounds = high_note_state.get_nr_of_play_sound()
                [notes.append(high_cardioid(start_time)) for _ in range(nr_of_sounds)]
            case PlaySound.LATE:
                start_time = time + (5 * random_range(0.05, 0.15))
                nr_of_sounds = high_note_state.get_nr_of_play_sound()
                [notes.append(high_cardioid(start_time)) for _ in range(nr_of_sounds)]
    else:
        match high_note_state.get_note_blend():
            case NoteBlend.ONLY_NOISE:
                notes.append(high_omni_noise(time, base_dur))
            case NoteBlend.ONLY_REVERB:
                notes.append(high_omni_reverb(time, base_dur))
            case NoteBlend.NOISE_AND_REVERB:
                notes.extend([
                    high_omni_noise(time, base_dur),
                    high_omni_reverb(time, base_dur)
                ])
        match high_note_state.get_play_sound():
            case PlaySound.EARLY:
                start_time = time + (3 * random_range(0.05, 0.15))
                nr_of_sounds = high_note_state.get_nr_of_play_sound()
                [notes.append(high_omni(start_time)) for _ in range(nr_of_sounds)]
            case PlaySound.LATE:
                start_time = time + (5 * random_range(0.05, 0.15))
                nr_of_sounds = high_note_state.get_nr_of_play_sound()
                [notes.append(high_omni(start_time)) for _ in range(nr_of_sounds)]
    return notes

high_sequencer = Sequencer(len(high_times)) \
    .add_step_handler(high_step_handler) \
    .start_time_handler(lambda time: time + (3 * random_range(0.05, 0.15))) \
    .next_time_handler(lambda i: high_note_state.get_note_start(middle_times[i][0]))


middle_note_state = NoteState(
    note_start_chain = MarkovChain(
        transition_matrix= {
            NoteStart.EARLY: {NoteStart.EARLY: 0.2, NoteStart.LATE: 0.8},
            NoteStart.LATE: {NoteStart.EARLY: 0.5, NoteStart.LATE: 0.5}
        }, initial_value=NoteStart.LATE),
    note_type_chain = MarkovChain(
        transition_matrix = {
            NoteTypeSwitch.CARIDOID: {NoteTypeSwitch.CARIDOID: 0.7, NoteTypeSwitch.OMNI: 0.3},
            NoteTypeSwitch.OMNI: {NoteTypeSwitch.CARIDOID: 0.7, NoteTypeSwitch.OMNI: 0.3},
        }, initial_value=NoteTypeSwitch.CARIDOID),
    note_blend_chain = MarkovChain(
        transition_matrix = {
            NoteBlend.ONLY_NOISE: {NoteBlend.ONLY_NOISE: 0.5, NoteBlend.ONLY_REVERB: 0.2, NoteBlend.NOISE_AND_REVERB: 0.3},
            NoteBlend.ONLY_REVERB: {NoteBlend.ONLY_NOISE: 0.8, NoteBlend.ONLY_REVERB: 0, NoteBlend.NOISE_AND_REVERB: 0.2},
            NoteBlend.NOISE_AND_REVERB: {NoteBlend.ONLY_NOISE: 0.8, NoteBlend.ONLY_REVERB: 0.2, NoteBlend.NOISE_AND_REVERB: 0.0},
        }, initial_value=NoteBlend.ONLY_NOISE),
    play_sound_chain = MarkovChain(
        transition_matrix= {
            PlaySound.EARLY: {PlaySound.EARLY: 0.0, PlaySound.LATE: 0.05, PlaySound.PAUSE: 0.95},
            PlaySound.LATE: {PlaySound.EARLY: 0.05, PlaySound.LATE: 0.0, PlaySound.PAUSE: 0.95},
            PlaySound.PAUSE: {PlaySound.EARLY: 0.05, PlaySound.LATE: 0.05, PlaySound.PAUSE: 0.9},
        }, initial_value=PlaySound.PAUSE),
    nr_of_play_sound_chanin=MarkovChain(
        transition_matrix= {
            1: {1: 0.5, 2: 0.5},
            2: {1: 0.9, 2: 0.1},
        }, initial_value=1)
)

middle_times = [(8, 13), (8, 13), (13, 13), (8, 13), (8, 13), (13, 13), (21, 13)] * 10

def middle_step_handler(i: int, time: float) -> list[SequenceNote]:
    notes = []
    base_dur = middle_times[i][1]
    if middle_note_state.get_note_type() == NoteTypeSwitch.CARIDOID:
        match middle_note_state.get_note_blend():
            case NoteBlend.ONLY_NOISE:
                notes.append(middle_cardioid_noise(time, base_dur))
            case NoteBlend.ONLY_REVERB:
                notes.append(middle_cardioid_reverb(time, base_dur))
            case NoteBlend.NOISE_AND_REVERB:
                notes.extend([
                    middle_cardioid_noise(time, base_dur),
                    middle_cardioid_reverb(time, base_dur)
                ])        
        match middle_note_state.get_play_sound():
            case PlaySound.EARLY:
                start_time = time + (3 * random_range(0.05, 0.15))
                nr_of_sounds = middle_note_state.get_nr_of_play_sound()
                [notes.append(middle_cardioid(start_time)) for _ in range(nr_of_sounds)]
            case PlaySound.LATE:
                start_time = time + (5 * random_range(0.05, 0.15))
                nr_of_sounds = middle_note_state.get_nr_of_play_sound()
                [notes.append(middle_cardioid(start_time)) for _ in range(nr_of_sounds)]

    else:
        match middle_note_state.get_note_blend():
            case NoteBlend.ONLY_NOISE:
                notes.append(middle_omni_noise(time, base_dur))
            case NoteBlend.ONLY_REVERB:
                notes.append(middle_omni_reverb(time, base_dur))
            case NoteBlend.NOISE_AND_REVERB:
                notes.extend([
                    middle_omni_noise(time, base_dur),
                    middle_omni_reverb(time, base_dur)
                ])
        match middle_note_state.get_play_sound():
            case PlaySound.EARLY:
                start_time = time + (3 * random_range(0.05, 0.15))
                nr_of_sounds = middle_note_state.get_nr_of_play_sound()
                [notes.append(middle_omni(start_time)) for _ in range(nr_of_sounds)]
            case PlaySound.LATE:
                start_time = time + (5 * random_range(0.05, 0.15))
                nr_of_sounds = middle_note_state.get_nr_of_play_sound()
                [notes.append(middle_omni(start_time)) for _ in range(nr_of_sounds)]
    return notes

middle_sequencer = Sequencer(len(middle_times)) \
    .add_step_handler(middle_step_handler) \
    .start_time_handler(lambda time: time + (3 * random_range(0.05, 0.15))) \
    .next_time_handler(lambda i: middle_note_state.get_note_start(middle_times[i][0])) \
    .spawn_sequencer(0, high_sequencer)

low_note_state = NoteState(
    note_start_chain = MarkovChain(
        transition_matrix= {
            NoteStart.EARLY: {NoteStart.EARLY: 0.2, NoteStart.LATE: 0.8},
            NoteStart.LATE: {NoteStart.EARLY: 0.5, NoteStart.LATE: 0.5}
        }, initial_value=NoteStart.LATE),
    note_type_chain = MarkovChain(
        transition_matrix = {
            NoteTypeSwitch.CARIDOID: {NoteTypeSwitch.CARIDOID: 0.7, NoteTypeSwitch.OMNI: 0.3},
            NoteTypeSwitch.OMNI: {NoteTypeSwitch.CARIDOID: 0.7, NoteTypeSwitch.OMNI: 0.3},
        }, initial_value=NoteTypeSwitch.CARIDOID),
    note_blend_chain = MarkovChain(
        transition_matrix = {
            NoteBlend.ONLY_NOISE: {NoteBlend.ONLY_NOISE: 0.5, NoteBlend.ONLY_REVERB: 0.2, NoteBlend.NOISE_AND_REVERB: 0.3},
            NoteBlend.ONLY_REVERB: {NoteBlend.ONLY_NOISE: 0.8, NoteBlend.ONLY_REVERB: 0, NoteBlend.NOISE_AND_REVERB: 0.2},
            NoteBlend.NOISE_AND_REVERB: {NoteBlend.ONLY_NOISE: 0.8, NoteBlend.ONLY_REVERB: 0.2, NoteBlend.NOISE_AND_REVERB: 0.0},
        }, initial_value=NoteBlend.ONLY_NOISE),
    play_sound_chain = MarkovChain(
        transition_matrix= {
            PlaySound.EARLY: {PlaySound.EARLY: 0.0, PlaySound.LATE: 0.05, PlaySound.PAUSE: 0.99},
            PlaySound.LATE: {PlaySound.EARLY: 0.05, PlaySound.LATE: 0.0, PlaySound.PAUSE: 0.95},
            PlaySound.PAUSE: {PlaySound.EARLY: 0.05, PlaySound.LATE: 0.05, PlaySound.PAUSE: 0.95},
        }, initial_value=PlaySound.PAUSE),
    nr_of_play_sound_chanin=MarkovChain(
        transition_matrix= {
            1: {1: 0.5, 2: 0.5},
            2: {1: 0.9, 2: 0.1},
        }, initial_value=1)
)

low_times = [(13, 13), (8, 13), (8, 13), (13, 13), (21, 13)] * 13

def low_step_handler(i: int, time: float) -> list[SequenceNote]:
    notes = []
    base_dur = low_times[i][1]
    if low_note_state.get_note_type() == NoteTypeSwitch.CARIDOID:
        match low_note_state.get_note_blend():
            case NoteBlend.ONLY_NOISE:
                notes.append(low_cardioid_noise(time, base_dur))
            case NoteBlend.ONLY_REVERB:
                notes.append(low_cardioid_reverb(time, base_dur))
            case NoteBlend.NOISE_AND_REVERB:
                notes.extend([
                    low_cardioid_noise(time, base_dur),
                    low_cardioid_reverb(time, base_dur)
                ])
        match low_note_state.get_play_sound():
            case PlaySound.EARLY:
                start_time = time + (3 * random_range(0.05, 0.15))
                nr_of_sounds = low_note_state.get_nr_of_play_sound()
                [notes.append(low_cardioid(start_time)) for _ in range(nr_of_sounds)]
            case PlaySound.LATE:
                start_time = time + (5 * random_range(0.05, 0.15))
                nr_of_sounds = low_note_state.get_nr_of_play_sound()
                [notes.append(low_cardioid(start_time)) for _ in range(nr_of_sounds)]        
        
    else:
        match low_note_state.get_note_blend():
            case NoteBlend.ONLY_NOISE:
                notes.append(low_omni_noise(time, base_dur))
            case NoteBlend.ONLY_REVERB:
                notes.append(low_omni_reverb(time, base_dur))
            case NoteBlend.NOISE_AND_REVERB:
                notes.extend([
                    low_omni_noise(time, base_dur),
                    low_omni_reverb(time, base_dur)
                ])
        match low_note_state.get_play_sound():
            case PlaySound.EARLY:
                start_time = time + (3 * random_range(0.05, 0.15))
                nr_of_sounds = low_note_state.get_nr_of_play_sound()
                [notes.append(low_omni(start_time)) for _ in range(nr_of_sounds)]
            case PlaySound.LATE:
                start_time = time + (5 * random_range(0.05, 0.15))
                nr_of_sounds = low_note_state.get_nr_of_play_sound()
                [notes.append(low_omni(start_time)) for _ in range(nr_of_sounds)]
        
    return notes

low_sequencer = Sequencer(len(low_times)) \
    .add_step_handler(low_step_handler) \
    .start_time_handler(lambda time: time + (3 * random_range(0.05, 0.15))) \
    .next_time_handler(lambda i: low_note_state.get_note_start(low_times[i][0])) \
    .spawn_sequencer(0, middle_sequencer)


sub_note_state = NoteState(
    note_start_chain=MarkovChain(
        transition_matrix= {
            NoteStart.EARLY: {NoteStart.EARLY: 0.2, NoteStart.LATE: 0.8},
            NoteStart.LATE: {NoteStart.EARLY: 0.5, NoteStart.LATE: 0.5}
        }, initial_value=NoteStart.LATE),
    note_type_chain=MarkovChain(
        transition_matrix = {
            NoteTypeSwitch.CARIDOID: {NoteTypeSwitch.CARIDOID: 0.7, NoteTypeSwitch.OMNI: 0.3},
            NoteTypeSwitch.OMNI: {NoteTypeSwitch.CARIDOID: 0.7, NoteTypeSwitch.OMNI: 0.3},
        }, initial_value=NoteTypeSwitch.CARIDOID),
    note_blend_chain=MarkovChain(
        transition_matrix = {
            NoteBlend.ONLY_NOISE: {NoteBlend.ONLY_NOISE: 0.5, NoteBlend.ONLY_REVERB: 0.2, NoteBlend.NOISE_AND_REVERB: 0.3},
            NoteBlend.ONLY_REVERB: {NoteBlend.ONLY_NOISE: 0.8, NoteBlend.ONLY_REVERB: 0, NoteBlend.NOISE_AND_REVERB: 0.2},
            NoteBlend.NOISE_AND_REVERB: {NoteBlend.ONLY_NOISE: 0.8, NoteBlend.ONLY_REVERB: 0.2, NoteBlend.NOISE_AND_REVERB: 0.0},
        }, initial_value=NoteBlend.ONLY_NOISE),
    play_sound_chain=MarkovChain(
        transition_matrix= {
            PlaySound.EARLY: {PlaySound.EARLY: 0.0, PlaySound.LATE: 0.05, PlaySound.PAUSE: 0.95},
            PlaySound.LATE: {PlaySound.EARLY: 0.05, PlaySound.LATE: 0.0, PlaySound.PAUSE: 0.95},
            PlaySound.PAUSE: {PlaySound.EARLY: 0.05, PlaySound.LATE: 0.05, PlaySound.PAUSE: 0.9},
        }, initial_value=PlaySound.PAUSE),
    nr_of_play_sound_chanin=MarkovChain(
        transition_matrix= {
            1: {1: 0.5, 2: 0.5},
            2: {1: 0.9, 2: 0.1},
        }, initial_value=1)
)

sub_times = [(13, 13), (8, 13), (13, 13), (21, 13)] * 14

def sub_step_handler(i: int, time: float) -> list[SequenceNote]:
    notes = []
    sub_dur = sub_times[i][1]
    match sub_note_state.get_note_blend():
        case NoteBlend.ONLY_NOISE:
            notes.append(sub_cardioid_noise(time, sub_dur))
        case NoteBlend.ONLY_REVERB:
            notes.append(sub_cardioid_reverb(time, sub_dur))
        case NoteBlend.NOISE_AND_REVERB:
            notes.extend([
                sub_cardioid_noise(time, sub_dur),
                sub_cardioid_reverb(time, sub_dur)
            ])
    match sub_note_state.get_play_sound():
        case PlaySound.EARLY:
            start_time = time + (3 * random_range(0.05, 0.15))
            nr_of_sounds = sub_note_state.get_nr_of_play_sound()
            [notes.append(sub_cardioid(start_time)) for _ in range(nr_of_sounds)]
        case PlaySound.LATE:
            start_time = time + (5 * random_range(0.05, 0.15))
            nr_of_sounds = sub_note_state.get_nr_of_play_sound()
            [notes.append(sub_cardioid(start_time)) for _ in range(nr_of_sounds)]

    return notes


sequence_notes = Sequencer(len(sub_times)) \
    .add_step_handler(sub_step_handler) \
    .next_time_handler(lambda i: sub_note_state.get_note_start(sub_times[i][0])) \
    .spawn_sequencer(0, low_sequencer) \
    .generate(0)

#sequence_notes = []

ui_piece = UiPieceBuilder() \
    .add_notes(sequence_notes) \
    .build()

piece_duration = ui_piece.get_duration()
min_freq, max_freq = ui_piece.get_track_min_max_freq()

piece_stats = { "total": piece_duration}

for track in ui_piece.tracks:
    track_duration = 0
    for note in track.notes:
        track_duration = max(track_duration, note.start + note.duration)
    piece_stats[track.track_name] = track_duration

display(piece_stats)

TRACK_HEIGHT = 200
NOTE_SCALE_FACTOR = 10
HEIGHT_INDENT = 180

ui_width = (200 + (piece_duration * NOTE_SCALE_FACTOR))
ui_height = (200 * len(ui_piece.tracks))

canvas = Canvas(width=ui_width, height=ui_height)
canvas.global_alpha = 0.7

for track_index, track in enumerate(ui_piece.tracks):
    canvas.font = "18px sans-serif"
    canvas.fill_style = "Black"
    canvas.fill_text(track.track_name, x=20, y=(track_index * TRACK_HEIGHT) + HEIGHT_INDENT)
    canvas.stroke_style = "Black"
    canvas.stroke_lines([(150, (track_index * TRACK_HEIGHT) + 10), (150, ((track_index * TRACK_HEIGHT) + TRACK_HEIGHT - 10))])
    for note in track.notes:        
        relative_note =  ((note.freq - min_freq) / (max_freq - min_freq))    
        startx = 200 + (note.start * NOTE_SCALE_FACTOR)        
        starty = (track_index * TRACK_HEIGHT) - (relative_note * HEIGHT_INDENT) + HEIGHT_INDENT
        peakx = 200 + (note.start + (note.duration * note.peak)) * NOTE_SCALE_FACTOR        
        peaky = (track_index * TRACK_HEIGHT) - (relative_note * HEIGHT_INDENT) + HEIGHT_INDENT - 5
        endx = 200 + (note.start + note.duration) * NOTE_SCALE_FACTOR
        endy = (track_index * TRACK_HEIGHT) - (relative_note * HEIGHT_INDENT) + HEIGHT_INDENT
        canvas.stroke_style = note.color
        canvas.stroke_lines([(startx, starty), (peakx, peaky), (endx, endy)])
canvas



{'total': 647.5677442319964,
 'Sub reverb': 557.0508342223457,
 'Low noise': 647.5677442319964,
 'Low reverb': 636.1078483730574,
 'Middle noise': 571.1590940050723,
 'Middle reverb': 551.5169987817349,
 'High noise': 530.5524123756373,
 'High reverb': 517.0840072320948,
 'High sound': 467.29889567363585,
 'Middle sound': 527.7863248170992,
 'Low sound': 284.87483712130563,
 'Sub noise': 553.3645198600914,
 'Sub sound': 541.2428012003877}

Canvas(height=2400, width=6675)

In [52]:
piece.stop()


In [2]:

def foo():
    return 5, 6

ret = foo()
print(type(ret))
a, b = foo()
print(a, b)

<class 'tuple'>
5 6
